# Import module

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random
import sympy 
import scipy.stats as st
import scipy.special
import itertools

import math
from decimal import *
from mpl_toolkits.mplot3d import Axes3D

In [2]:
class switch(object):
    def __init__(self, value):
        self.value = value
        self.fall = False
    def __iter__(self):
        """Return the match method once, then stop"""
        yield self.match
        raise StopIteration  
    def match(self, *args):
        """Indicate whether or not to enter a case suite"""
        if self.fall or not args:
            return True
        elif self.value in args: # changed for v1.5, see below
            self.fall = True
            return True
        else:
            return False

## UE_list
UE_list[num_UE][0]=arrival time

UE_list[num_UE][1]=attempt 1st time

UE_list[num_UE][2]=attempt 2nd time(default -1)

UE_list[num_UE][3]=attempt 3rd time(default -1)

UE_list[num_UE][4]=attempt 4th time(default -1)

UE_list[num_UE][5]=status(success(1)/fail(0)/trying(2))

In [3]:
class init_RAparameter():
    def __init__(self,config,sample): 
        self.config = config 
        self.sample = sample
        self.nUE = num_UE
        self.subcarrier = R
        self.success_time = 0
        self.P_s =  0
        self.iteNiS = [0 for _ in range(0,maxslot())]
        self.iteNiF = [0 for _ in range(0,maxslot())]
        self.iteNiR = [0 for _ in range(0,maxslot())]
        self.iteNiI = [0 for _ in range(0,maxslot())]
        self.iteMi = [[0]*(retrans+1) for _ in range(0,maxslot())]
        self.iteMiSn = [[0]*(retrans+1) for _ in range(0,maxslot())]
        self.iteMiFn = [[0]*(retrans+1) for _ in range(0,maxslot())]
        self.totalMis = [[0]*(maxslot()+1) for _ in range(0,sample)]
        self.resrNum = [0 for j in range(0,maxslot())]
        self.M1S = [0 for _ in range(0,sample)] # for testing (HJR)
        self.CDF = [0 for _ in range(0,retrans+1)]
        self.Da = 0
        self.Da_MiSn = 0
        self.DaSn = [0]*maxslot() #Da/transmission
    def define_RAparameter(self):
        self.dict_retr = gen_UE_id(self.nUE)
        self.dict_prea = gen_UE_prea(self.nUE)
        self.slot_list = [[]for j in range(0,maxslot())]
        self.dict_backoffcnt = gen_UE_backoffcnt(self.nUE)
        
        # NFRP Features
        if NFRP_RA == 1:
            self.dict_NFRPbackoffcnt = gen_UE_NFRPbackoffcnt(self.nUE)
        # ---
        
        self.NiS = [0 for _ in range(0,maxslot())]
        self.NiF = [0 for _ in range(0,maxslot())]
        self.NiR = [0 for _ in range(0,maxslot())]
        self.NiI = [0 for _ in range(0,maxslot())]
        self.Mi = [[0]*(retrans+1) for _ in range(0,maxslot())]
        self.MiSn = [[0]*(retrans+1) for _ in range(0,maxslot())]
        self.MiFn = [[0]*(retrans+1) for _ in range(0,maxslot())]
        self.T = [0 for j in range(0,maxslot())]
        self.UE_list=[[-1]*(retrans+3) for _ in range(0,self.nUE)] #retrans+3?
        self.arrive_num = [0 for j in range(0,maxslot())]
    def UORA_departure_process(self,slot,countdown,ite): ## UORA countdown RA-RU here
        for ue in list(self.dict_retr):
            self.dict_backoffcnt[ue] -= countdown
            if self.dict_backoffcnt[ue] <= 0:  ## can contend for the RA-RU here
                # genearate preamble
                retry = self.dict_retr[ue]
                self.Mi[slot][retry] += 1
                self.dict_prea[ue]= doRA(self,slot) ## pick random RA-RU here
                self.slot_list[slot].append(ue) ## append the ue to the slot_list for recording the M_i?  
  
        # check preamble
        self.UORA_check_preamble(slot,self.subcarrier,ite)
        ## add retransmission round
        doretrans(self.dict_retr,self.slot_list[slot],self.UE_list)  ## dict_retr, retr_ue, unused UE_list                
        ## do random backoff for fail devices    
        backoff(self.dict_retr,self.dict_backoffcnt,self.slot_list,slot)
        
    def NFRP_departure_process(self,slot,countdown,ite): ## UORA countdown RA-RU here
        for ue in list(self.dict_retr):
            self.dict_NFRPbackoffcnt[ue] -= countdown
            if self.dict_NFRPbackoffcnt[ue] <= 0:  ## can contend for the RA-RU here
                # genearate preamble
                retry = self.dict_retr[ue]
                self.Mi[slot][retry] += 1
                self.dict_prea[ue]= doNFRPRA(self,slot) ## pick random RA-RU here
                self.slot_list[slot].append(ue) ## append the ue to the slot_list for recording the M_i? 

        # check preamble
        self.NFRP_check_preamble(slot,self.subcarrier,ite)
        ## add retransmission round
        doretrans(self.dict_retr,self.slot_list[slot],self.UE_list)  ## dict_retr, retr_ue, unused UE_list                
        ## do random backoff for fail devices    
        NFRPbackoff(self.dict_retr,self.dict_NFRPbackoffcnt,self.slot_list,slot)

    def UORA_check_preamble(self,slot,subcarrier,ite):
        for preamble in range(1,subcarrier+1):
            ## output UE_id transmitting same preamble
            collided_ue = preak(self.dict_prea, self.slot_list[slot], preamble) 
            ## Input collided_ue, output success or not -> collided_ue!=0 表示有大於1的ue選擇這個preamble 
            if len(collided_ue) != 0:                                              
                find_succuss_conv_RA(self,slot,collided_ue,ite)
        
    def NFRP_check_preamble(self,slot,subcarrier,ite):
        used_preamble = []
        removed_preamble = []
        allocated = []
        
        if individual_tones == 0:
            for preamble in range(1,N+1):
                collided_ue_NFRP = preak_NFRP(self.dict_prea, self.slot_list[slot], preamble)       
                if len(collided_ue_NFRP) == 1:
                    used_preamble.append(preamble)
                    self.NiS[slot] += 1
                elif len(collided_ue_NFRP) > 1:
                    feedback_status_0 = 0
                    feedback_status_1 = 0
                    for i in collided_ue_NFRP:
                        if self.dict_prea[i][1] == 0:
                            feedback_status_0 = 1
                        elif self.dict_prea[i][1] == 1:
                            feedback_status_1 = 1
                    if feedback_status_0 == 1 and feedback_status_1 == 1:
                        removed_preamble.append(preamble)
                        self.NiF[slot] += 1
                    if feedback_status_0 == 1 and feedback_status_1 == 0:
                        used_preamble.append(preamble)
                        self.NiS[slot] += 1
                    if feedback_status_0 == 0 and feedback_status_1 == 1:
                        used_preamble.append(preamble)
                        self.NiS[slot] += 1
            self.NiI[slot] = N - self.NiS[slot] - self.NiF[slot]                   

            # print("Used preamble", used_preamble)
            # print("Removed preamble", removed_preamble)

            if len(used_preamble)<= R:
                allocated = used_preamble
            else:
                allocated = np.random.choice(used_preamble, size=R, replace=False)
            
            # print("Allocated preamble", allocated)
            
            for preamble in range(1,N+1):
                failed = 0
                collided_ue_NFRP = preak_NFRP(self.dict_prea, self.slot_list[slot], preamble)
                if len(collided_ue_NFRP) != 0:
                    for NDP_RU in allocated:
                        if preamble == NDP_RU:
                            find_succuss_conv_RA_NFRP(self,slot,collided_ue_NFRP,ite)
                            failed = 0
                            break
                        else:
                            failed = 1
                    if failed == 1:
                        failed_conv_RA_NFRP(self,slot,collided_ue_NFRP,ite)
                
        else:
            for preamble in range(1,N*2+1):
                collided_ue_NFRP = preak_NFRP(self.dict_prea, self.slot_list[slot], preamble) 
                if len(collided_ue_NFRP) == 1:
                    used_preamble.append(preamble)
                    self.NiS[slot] += 1
                if len(collided_ue_NFRP) > 1:
                    used_preamble.append(preamble)
                    self.NiS[slot] += 1
            self.NiI[slot] = N*2 - self.NiS[slot]
                    
            # print("Used preamble", used_preamble)
            if len(used_preamble)<= R:
                allocated = used_preamble
            else:
                allocated = np.random.choice(used_preamble, size=R, replace=False)
            # print("Allocated preamble", allocated)
            
            for preamble in range(1,N*2+1):
                failed = 0
                collided_ue_NFRP = preak_NFRP(self.dict_prea, self.slot_list[slot], preamble)
                if len(collided_ue_NFRP) != 0:
                    for NDP_RU in allocated:
                        if preamble == NDP_RU:
                            find_succuss_conv_RA_NFRP(self,slot,collided_ue_NFRP,ite)
                            failed = 0
                            break
                        else:
                            failed = 1
                    if failed == 1:
                        failed_conv_RA_NFRP(self,slot,collided_ue_NFRP,ite)

In [4]:
class init_UORAparameter(init_RAparameter):
    def __init__(self,config,sample):
        super().__init__(config,sample) #init_RAparameter?
        self.itereserveRARU = [0 for _ in range(0,maxslot())] #bug for RARU is larger then Rinitialize array with 0
    def define_RAparameter(self):
        super().define_RAparameter()
        self.reserveRARU = [R for _ in range(0,maxslot())] #intialize array with R RA-RUs

#         print('maxslot',maxslot(),'K',K)

    def calculate_statistic(self,ite): # testing
        for temp_i in range(0,maxslot()):
            self.itereserveRARU[temp_i] += self.reserveRARU[temp_i]
            self.iteNiS[temp_i] += self.NiS[temp_i]
            self.iteNiF[temp_i] += self.NiF[temp_i]
            self.iteNiR[temp_i] += self.NiR[temp_i]
            self.iteNiI[temp_i] += self.NiI[temp_i]
            for temp_n in range (0,retrans+1):
                self.iteMi[temp_i][temp_n] += self.Mi[temp_i][temp_n]
                self.iteMiSn[temp_i][temp_n] += self.MiSn[temp_i][temp_n] 
                self.iteMiFn[temp_i][temp_n] += self.MiFn[temp_i][temp_n]
#                 if temp_i == 0: # testing
#                     self.M1S[ite] += self.MiSn[temp_i][temp_n]
                
    def report(self):
        #Access prob ------------------------------------------------------
        MI = 0
        MS = 0
        temp_Mis = [0 for _ in range(0,maxslot())] # (HJR Da)
#         print('TempRARU:', self.itereserveRARU,'\n')
        for temp_i in range(1,maxslot()):
            self.itereserveRARU[temp_i] = math.floor((self.itereserveRARU[temp_i]/float(self.sample))*1000000)/1000000
            self.iteNiS[temp_i] = math.floor((self.iteNiS[temp_i]/float(self.sample))*1000000)/1000000
            self.iteNiF[temp_i] = math.floor((self.iteNiF[temp_i]/float(self.sample))*1000000)/1000000
            self.iteNiR[temp_i] = math.floor((self.iteNiR[temp_i]/float(self.sample))*1000000)/1000000
            self.iteNiI[temp_i] = math.floor((self.iteNiI[temp_i]/float(self.sample))*1000000)/1000000
            for temp_n in range (0,retrans+1):
                self.iteMi[temp_i][temp_n] = self.iteMi[temp_i][temp_n]/float(self.sample)
                self.iteMiSn[temp_i][temp_n] = self.iteMiSn[temp_i][temp_n]/float(self.sample) #adv
                self.iteMiFn[temp_i][temp_n] = self.iteMiFn[temp_i][temp_n]/float(self.sample) #adv
#                 MI += self.iteMi[temp_i][temp_n]
                MS += self.iteMiSn[temp_i][temp_n]
#         print('iteMi:',self.iteMi)
#         print(self.iteMiSn)
#         plotMiSn(self.iteMiSn)
        MS = round((MS)*1000000)/1000000 
        if self.nUE!=0:
            self.P_s = round((float(MS)/float(self.nUE))*1000000)/1000000
#             print('Access prob ',self.P_s)
            #print(self.P_s)
        else:
            self.P_s = 0.0

        #Utilization ------------------------------------------------------
        Uti = 0
        if R!=0:
            Uti = round( (float(MS)/float(maxslot()*R) )*1000000)/1000000
        #print('Utilization ',Uti)

        #CDF --------------------------------------------------------------
#         MS_K = [0 for _ in range (0,retrans+1)]
#         MS_K_temp = 0
#         ##print('CDF , F( m )')
#         for temp_n in range(0,retrans+1): # k = temp_n +1
#             for temp_i in range(0,maxslot()):
#                 #for temp_n in range (0,k):
#                 #MS_k[temp_n] += iteMiSn[temp_i][temp_n]  
                
#                 MS_K_temp += self.iteMiSn[temp_i][temp_n]*K
#             MS_K[temp_n] = MS_K_temp
#            # print('MS_K',MS_K)
#             MS_K[temp_n] = math.floor((MS_K[temp_n])*1000000)/1000000
#            # print('MS_K',MS_K)
#             #print('      F(',temp_n+1,')', math.floor((MS_K/MS)*1000)/1000 ) #MS_k[temp_n]/MS)
#         for temp_n in range(0,retrans+1):   
#             #print('MS_K[temp_n]',MS_K[temp_n],'\n')
#             #print('MS_K[retrans]',MS_K[retrans],'\n')
#             self.CDF[temp_n] = math.floor((MS_K[temp_n]/MS_K[retrans])*1000000)/1000000
            
        #Access delay  DA -------------------------------------------------
        for temp_i in range(1,maxslot()):
            for temp_n in range (0,retrans+1):    # count Mis
                temp_Mis[temp_i] += self.iteMiSn[temp_i][temp_n]
                
            for j in range(temp_i+1+1,temp_i+int(np.ceil(temp_Mis[temp_i]*(K-1)/R))+1):  # modify part (HJR)
                self.T[temp_i] += j*R               # modify part (HJR)
#             self.T[temp_i]=(temp_i)+K
#             print(self.T[temp_i])
#                 self.Da_MiSn+=self.iteMiSn[temp_i][temp_n]*float(self.T[temp_i])
            self.Da_MiSn+=temp_Mis[temp_i]*temp_i + float(self.T[temp_i]) + (temp_Mis[temp_i]*(K-1)%R)*(temp_i+int(np.ceil(temp_Mis[temp_i]*(K-1)/R))+1)#分子
            self.DaSn[temp_i] = self.DaSn[temp_i-1] + temp_Mis[temp_i]*temp_i + float(self.T[temp_i]) + (temp_Mis[temp_i]*(K-1)%R)*(temp_i+int(np.ceil(temp_Mis[temp_i]*(K-1)/R))+1)            
#             self.Da_MiSn+=temp_Mis[temp_i]*float(self.T[temp_i]) #分子
#             self.Da_MiSn+=temp_Mis[temp_i]*float(self.T[temp_i]) #分子
        if self.nUE!=0:
            
            if(NFRP_RA != 1):
                self.Da= float(self.Da_MiSn*T_UORA) / float(MS*K) 
            else:
                self.Da= float(self.Da_MiSn*T_NFRP) / float(MS*K) 
            
            sum_temp_Mis = 0
            for temp_i in range(1,maxslot()):
                sum_temp_Mis += temp_Mis[temp_i]
                if(temp_Mis[temp_i]==0):
                    self.DaSn[temp_i] = self.DaSn[temp_i-1]
                else:
                    if(NFRP_RA == 1):
                        self.DaSn[temp_i] = float(self.DaSn[temp_i]*T_NFRP) / float(sum_temp_Mis*K)
                    else:
                        self.DaSn[temp_i] = float(self.DaSn[temp_i]*T_UORA) / float(sum_temp_Mis*K)
#             print("DaSn",self.DaSn[temp_i])
            
        else:
            self.Da = 0.0
        #print('Da=',self.Da)  
            
        

# Main Function

- main()
    - initail statistical value
    - for loop (sample)
        - initial dictionary for diff case
        - for loop (slot)
            - arrival_process()
            - departue_process()
                - for loop (UE) 
                    - doRA()
                - check_preamble()
                - doretrans()
                - backoff()
    - return the success probability




In [5]:
def main(sample):

    # define parameter
    for case in switch(CONFIG):
        if case('UORA'):    
            cUORA = init_UORAparameter("UORA",sample)
            cobject = cUORA
            break
        if case():
            print('CONFIG ERROR at beginning')
            break
    
    # sample loop
    for ite in range(0,cobject.sample):                       
        
        # initial parameter
        cobject.define_RAparameter()
        
        # slot loop
        for slot in range(0,len(cobject.slot_list)):
            
            # arrival process
            arrival_process(slot,cobject)
            
            # departure process
            tag = 0
#             print('slot',slot)
            for case in switch(CONFIG):
                if case('UORA'):
                    #if in dict_retr still has sta didnt reach to its retrans limit than excute departure process
                    if len(cobject.dict_retr)>0:
                        if NFRP_RA == 1 and TRA == 1:
                            if individual_tones == 1:
                                cUORA.NFRP_departure_process(slot+1,N*2,ite)
                            else:
                                cUORA.NFRP_departure_process(slot+1,N,ite)
                            cUORA.UORA_departure_process(slot+1,cUORA.reserveRARU[slot+1],ite) 
                        if NFRP_RA == 1:
                            if individual_tones == 1:
                                cUORA.NFRP_departure_process(slot+1,N*2,ite)
                            else:
                                cUORA.NFRP_departure_process(slot+1,N,ite)
                        else:
                            cUORA.UORA_departure_process(slot+1,cUORA.reserveRARU[slot+1],ite) 
                        break
                    else:
                        tag = 1
                if case():
                    #print('CONFIG ERROR at slot loop')
                    break
            if tag == 1:
                if NFRP_RA == 1:
                    if individual_tones == 1:
                        for i in range (slot+1,maxslot()):
                            cUORA.NiI[i] = N*2
                    else:    
                        for i in range (slot+1,maxslot()):
                            cUORA.NiI[i] = N
                break
        cobject.calculate_statistic(ite)
        
        
    # return vaule (success probability)
    if cobject.nUE != 0:
        #print_UE_list(cobject)
        cobject.report()
        #return cobject.Da
#         return cobject.P_s
        return cobject
    else:
        return -1

## Initail Function

- gen_UE_id()
    - give UE id for each UE
- gen_UE_prea()
    - initail the dict_prea list
- gen_UE_beam()
    - initail the dict_beam list
- maxslot()
    - calculate the Imax
- gen_UE_posi()
    - give random position
- gen_UE_tx()
    - give a msg3 transmission time
- gen_UE_L()
    - generate the packet length for each ue
- compare_L_and_TBS()
    - compare the packet lenght with TBS



In [6]:
def gen_UE_id(num_UE):                        ## Give UE id for each UE ##
    seq = [i for i in range(0,num_UE)]
    dict_id = dict.fromkeys(seq,0)
    return dict_id

def gen_UE_prea(num_UE):                      ## Initial dict_prea list 紀錄每個ＵＥ所選到的preamble##
    dict_prea = {}
    for i in range(0,num_UE):
        dict_prea[i] = 0
    return dict_prea

def gen_UE_backoffcnt(num_UE):
    dict_backoffcnt = {}
    for case in switch(CONFIG):
        if case('UORA'):#only for WiFi         
            for i in range(0,num_UE):
                dict_backoffcnt[i] = int(random.uniform(0,UORA_BO[0]+1))#設定init OBO
#                 print('STA:',i,'OBO',dict_backoffcnt[i])
            break
        if case():#conv_RA...TARA...
            for i in range(0,num_UE):
                dict_backoffcnt[i] = 0
            break   

    return dict_backoffcnt

# NFRP Features
def gen_UE_NFRPbackoffcnt(num_UE):
    dict_NFRPbackoffcnt = {}
    for case in switch(CONFIG):
        if NFRP_RA == 1:     
            for i in range(0,num_UE):
                dict_NFRPbackoffcnt[i] = int(random.uniform(0,NFRP_RA_BO[0]+1))#設定init OBO
#                 print('STA:',i,'OBO',dict_backoffcnt[i])
            break

    return dict_NFRPbackoffcnt
# ---

def maxslot():
    for case in switch(CONFIG):
        if case('UORA'):#only for WiFi
            Maxraslot = 50
            break  
        if case():#conv_RA...TARA...
            Maxraslot = np.ceil((T_RAR+W_RAR+W_BO)/T_RAREP)*(retrans+1)+1 #np.ceil 向上取整
            break            
    return int(Maxraslot)

def normal(mean, std):
    u = random.random()
    v = random.random()
    return abs((np.sqrt(-2 * np.log(u)) * np.cos(2 * np.pi * v) * std) + mean)

## Arrival Process
- TODO: need to record every arrival ue/STA's slot and update it in dict_retr(?)

In [7]:
def arrival_process(slot,cobject):
    #one-shot
    if slot==0 :         
        if(CONFIG=='UORA'):
            for i in range (0,cobject.nUE):
                # NFRP-RA
                if(NFRP_RA == 1):
                    init_obo = int(random.uniform(0,NFRP_RA_BO[0]+1))# update initOBO
                    cobject.dict_NFRPbackoffcnt[i] = init_obo #設定OBO
                # ---
                
                init_obo = int(random.uniform(0,UORA_BO[0]+1))# update initOBO
                cobject.dict_backoffcnt[i] = init_obo #設定OBO
        else:
            cobject.dict_retr = gen_UE_id(cobject.nUE)
            cobject.arrive_num[slot]=len(cobject.dict_retr)
            for ue_id in cobject.dict_retr:
                # Save info in the UE_list
                cobject.UE_list[ue_id][0] = 0
            for i in range (0,cobject.nUE):
                cobject.dict_backoffcnt[i] = 1
    else:
        cobject.arrive_num[slot]=len(cobject.dict_retr)
        # Here can process other arrival process,ex: Possion

## Departure Process

- doRA()
- doNFRPRA()

In [8]:
def doRA(self,slot):
    if self.reserveRARU[slot]!=0:    
        return random.randint(1,self.reserveRARU[slot])
    else:
        return -1

In [9]:
def doNFRPRA(self,slot):
    if NFRP_RA == 1:
        if individual_tones == 0:
            NFRP_RA_Preamble = [0]*2
            NFRP_RA_Preamble[0] = random.randint(1,N)
            NFRP_RA_Preamble[1] = random.randint(0,1)
            return NFRP_RA_Preamble
        else:
            return random.randint(1,N*2)
    else:
        return -1

- preak()
- preak_NFRP()

In [10]:
def preak(dict_prea, ue_list, preamble):
    out = []
    for i in ue_list:
        #print(i)
        if dict_prea[i] == preamble: #檢查在slot_list[slot]的ＵＥ有幾個等於preamble
            out.append(i) #如果有的話out list增加,放有選到該raru的sta編號
    return out

def preak_NFRP(dict_prea, ue_list, preamble):
    out = []

    for i in ue_list:
        #print(i)
        if individual_tones == 0:
            if dict_prea[i][0] == preamble: 
                out.append(i) 
        else:
            if dict_prea[i] == preamble: 
                out.append(i) 
    return out

In [11]:
def find_succuss_conv_RA(self,slot,collided_ue,ite):
    if len(collided_ue) == 1 : ## only one STA picks this preamble/RA-RU
        result = collided_ue[0]   #success
        self.totalMis[ite][slot] += 1
        #check = check+1 ## marked success
        retry = self.dict_retr[result]
        self.resrNum[slot] += K-1
        self.MiSn[slot][retry] += 1 #add one success in MiSn
#         self.success_time = self.success_time + (slot)*T_RAREP + T_RAR + W_RAR ## not for UORA?? only TXOP?
        self.success_time = self.success_time + slot*T_UORA
#         print('collided_ue[0]:',collided_ue[0])
        tempK = K-1
        tempi = 1
        while tempK != 0 :
#             print('slot:',slot,'tempK:',tempK)
#             print('slot+tempi:',slot+tempi,'reserveRARU:', self.reserveRARU[slot+tempi])
            if self.reserveRARU[slot+tempi] >= tempK :
                self.reserveRARU[slot+tempi] -= tempK
                tempK = 0
#                 print('tempK:?', tempK)
            elif self.reserveRARU[slot+tempi] < tempK and self.reserveRARU[slot+tempi] != 0 :
                tempK -= self.reserveRARU[slot+tempi]
                self.reserveRARU[slot+tempi] = 0
                tempi += 1
            elif self.reserveRARU[slot+tempi] == 0 :
                tempi += 1
                
#         for k_temp in range(1,K): ## success STA will occupy the RU for k times.
#             print('slot+k_temp',slot+k_temp)
#             self.reserveRARU[slot+k_temp] -= 1 ## minus the further RA-RU
#             print('slot',slot,'slot+k_temp',slot+k_temp,'reserve',self.reserveRARU[slot+k_temp]) #case2 debug
#         self.UE_list[result][5]=1

        del self.dict_retr[result]                 ## the ue transmit success will be remove from dictionary    
    else:
        for temp in collided_ue:
            retry = self.dict_retr[temp]
            self.MiFn[slot][retry] +=1
#             self.UE_list[temp][5]=0
        result = -1

def find_succuss_conv_RA_NFRP(self,slot,collided_ue,ite):
    if len(collided_ue) == 1 : ## only one STA picks this preamble/RA-RU
        result = collided_ue[0]   #success
        self.totalMis[ite][slot] += 1
        #check = check+1 ## marked success
        retry = self.dict_retr[result]
        self.resrNum[slot] += K-1
        self.MiSn[slot][retry] += 1 #add one success in MiSn
#         self.success_time = self.success_time + (slot)*T_RAREP + T_RAR + W_RAR ## not for UORA?? only TXOP?
        self.success_time = self.success_time + slot*T_UORA
#         print('collided_ue[0]:',collided_ue[0])
        tempK = K-1
        tempi = 1
        while tempK != 0 :
#             print('slot:',slot,'tempK:',tempK)
#             print('slot+tempi:',slot+tempi,'reserveRARU:', self.reserveRARU[slot+tempi])
            if self.reserveRARU[slot+tempi] >= tempK :
                self.reserveRARU[slot+tempi] -= tempK
                tempK = 0
#                 print('tempK:?', tempK)
            elif self.reserveRARU[slot+tempi] < tempK and self.reserveRARU[slot+tempi] != 0 :
                tempK -= self.reserveRARU[slot+tempi]
                self.reserveRARU[slot+tempi] = 0
                tempi += 1
            elif self.reserveRARU[slot+tempi] == 0 :
                tempi += 1
                
#         for k_temp in range(1,K): ## success STA will occupy the RU for k times.
#             print('slot+k_temp',slot+k_temp)
#             self.reserveRARU[slot+k_temp] -= 1 ## minus the further RA-RU
#             print('slot',slot,'slot+k_temp',slot+k_temp,'reserve',self.reserveRARU[slot+k_temp]) #case2 debug
#         self.UE_list[result][5]=1

        del self.dict_retr[result]                 ## the ue transmit success will be remove from dictionary    
    else:
        for temp in collided_ue:
            retry = self.dict_retr[temp]
            self.MiFn[slot][retry] +=1
#             self.UE_list[temp][5]=0
        result = -1
    
def failed_conv_RA_NFRP(self,slot,collided_ue,ite):
    if len(collided_ue) == 1 : ## only one STA picks this preamble/RA-RU
        result = collided_ue[0]
        retry = self.dict_retr[result]
        self.MiFn[slot][retry] += 1 
        result = -1
    else:
        for temp in collided_ue:
            retry = self.dict_retr[temp]
            self.MiFn[slot][retry] +=1
    #             self.UE_list[temp][5]=0
        result = -1

### Retransmission and Backoff Function
- doretrans()
- backoff()
- NFRPbackoff()


In [12]:
def doretrans(dict_retr,retr_ue,UE_list):
    for i in retr_ue:        
        if i in dict_retr:            
            dict_retr[i] += 1
#             UE_list[i][retrans+1]=2
            #print('ue:',i,'after++ dict_retr[ue]:',dict_retr[i])#debug
            if dict_retr[i] > retrans:
               # print('ue',i,'rich max retr')#DEBUG
#                 UE_list[i][retrans+1]=0
                del dict_retr[i]
    return 0


def backoff(dict_retr,dict_backoffcnt,slot_list,slot):
    for case in switch(CONFIG):
        if case('UORA'):
            for i in slot_list[slot]:
                if i in dict_retr:
                    dict_backoffcnt[i] = int(random.uniform(0,UORA_BO[dict_retr[i]]+1))
            break
        if case():
            for i in slot_list[slot]:
                if i in dict_retr:
                    retrans_slot = int(slot + np.ceil((T_RAR+W_RAR+random.uniform(0,W_BO))/T_RAREP))
                    dict_backoffcnt[i]= retrans_slot - slot
            break               
    return 0   

def NFRPbackoff(dict_retr,dict_NFRPbackoffcnt,slot_list,slot):
    for case in switch(CONFIG):
        if case('UORA'):
            if NFRP_RA == 1:
                for i in slot_list[slot]:
                    if i in dict_retr:
                        dict_NFRPbackoffcnt[i] = int(random.uniform(0,NFRP_RA_BO[dict_retr[i]]+1))
            break              
    return 0    

## WiFi Configuration

In [13]:
# In miliseconds
T_PIFS = 0.025
T_TF = 0.1
T_SIFS = 0.016
T_RU = 0.8 # 10 kbpms
T_MBA = 0.032
T_NDP = 0.072

T_UORA = T_PIFS + T_TF + 2*T_SIFS + T_RU + T_MBA
T_NFRP = T_PIFS + T_TF + T_SIFS + T_NDP + T_TF + 3*T_SIFS + T_RU + T_MBA

---
# Test Result

In [14]:
sample = 100000
CONFIG = 'UORA' #fixed-RA
ARRIVAL = 'one-shot'
R = 18 # number of RA-RU
K = 1
retrans = 0
num_UE_g = [1,2,3,4,5,6,7,8,9,10,20,30,40,50,60,70,80,90,100]
UORA_BO =  [7,15,31,31,31] #OCWmin~max [7,15,31,31,31]

# NFRP Features
NFRP_RA = 0
TRA = 0
individual_tones = 0

N = 36
NFRP_RA_BO = [7,15,31,31,31]

print('Number of samples=',sample)
print('Number of RUs=',R)
print('K=',K)
print('Total transmissions=', retrans+1,'\n')

print('Features:')
if NFRP_RA == 1 and TRA == 1 and individual_tones == 1:
    print('NFPR-enhanced UORA enabled')
    print('Number of NDP Sets=', N*2)
elif NFRP_RA == 1 and TRA == 1 and individual_tones == 0:
    print('NFPR-enhanced UORA enabled')
    print('Number of NDP Sets=', N)
elif NFRP_RA == 1 and TRA == 0 and individual_tones == 1:
    print('NFRP-RA enabled')
    print('Number of NDP Sets=', N*2)
elif NFRP_RA == 1 and TRA == 0 and individual_tones == 0:
    print('NFRP-RA enabled')
    print('Number of NDP Sets=', N)
        
print('<<--------------------------->>\n')

for num_UE in num_UE_g:
    if sample > 0:
        cobject = main(sample)
        
        print('# Number of UE:', num_UE)
        print('P_s:',cobject.P_s,'Da:',cobject.Da,'\n')
        
    #     print('Duration:', cobject.success_time,'\n')
        
        print('Mi:',cobject.iteMi,'\n')
        print('MiSn:',cobject.iteMiSn,'\n')
    
print('end sim')

Number of samples= 0
Number of RUs= 18
K= 1
Total transmissions= 5 

Features:
<<--------------------------->>

end sim


In [15]:
sample = 0
CONFIG = 'UORA' #fixed-RA
ARRIVAL = 'one-shot'
R = 18 # number of RA-RU
K = 1
retrans = 4
num_UE_g = [10,20,30,40,50,60,70,80,90,100]
UORA_BO =  [7,15,31,31,31] #OCWmin~max [7,15,31,31,31]

# NFRP Features
NFRP_RA = 1
TRA = 0
individual_tones = 0

N = 36
NFRP_RA_BO = [7,15,31,31,31]

print('Number of samples=',sample)
print('Number of RUs=',R)
print('K=',K)
print('Total transmissions=', retrans+1,'\n')

print('Features:')
if NFRP_RA == 1 and TRA == 1 and individual_tones == 1:
    print('NFPR-enhanced UORA enabled')
    print('Number of NDP Sets=', N*2)
elif NFRP_RA == 1 and TRA == 1 and individual_tones == 0:
    print('NFPR-enhanced UORA enabled')
    print('Number of NDP Sets=', N)
elif NFRP_RA == 1 and TRA == 0 and individual_tones == 1:
    print('NFRP-RA enabled')
    print('Number of NDP Sets=', N*2)
elif NFRP_RA == 1 and TRA == 0 and individual_tones == 0:
    print('NFRP-RA enabled')
    print('Number of NDP Sets=', N)
        
print('<<--------------------------->>\n')

for num_UE in num_UE_g:
    if sample > 0:
        cobject = main(sample)
        
        print('# Number of UE:', num_UE)
        print('P_s:',cobject.P_s,'Da:',cobject.Da,'\n')
        
        print('Mi:',cobject.iteMi,'\n')
        print('MiSn:',cobject.iteMiSn,'\n')

        print('NiS:',cobject.iteNiS,'\n')
        print('NiF:',cobject.iteNiF,'\n')
        print('NiI:',cobject.iteNiI,'\n')

print('end sim')

Number of samples= 100000
Number of RUs= 18
K= 1
Total transmissions= 5 

Features:
NFRP-RA enabled
Number of NDP Sets= 36
<<--------------------------->>

# Number of UE: 10
P_s: 0.999996 Da: 1.4790002270009084 

Mi: [[0, 0, 0, 0, 0], [10.0, 0.0, 0.0, 0.0, 0.0], [0.0, 2.24032, 0.0, 0.0, 0.0], [0.0, 0.0, 0.15251, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0044, 0.0], [0.0, 0.0, 0.0, 0.0, 0.00024], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0

In [16]:
sample = 0
CONFIG = 'UORA' #fixed-RA
ARRIVAL = 'one-shot'
R = 18 # number of RA-RU
K = 1
retrans = 4
num_UE_g = [10,20,30,40,50,60,70,80,90,100]
UORA_BO =  [7,15,31,31,31] #OCWmin~max [7,15,31,31,31]

# NFRP Features
NFRP_RA = 1
TRA = 0
individual_tones = 1

N = 36
NFRP_RA_BO = [7,15,31,31,31]

print('Number of samples=',sample)
print('Number of RUs=',R)
print('K=',K)
print('Total transmissions=', retrans+1,'\n')

print('Features:')
if NFRP_RA == 1 and TRA == 1 and individual_tones == 1:
    print('NFPR-enhanced UORA enabled')
    print('Number of NDP Sets=', N*2)
elif NFRP_RA == 1 and TRA == 1 and individual_tones == 0:
    print('NFPR-enhanced UORA enabled')
    print('Number of NDP Sets=', N)
elif NFRP_RA == 1 and TRA == 0 and individual_tones == 1:
    print('NFRP-RA enabled')
    print('Number of NDP Sets=', N*2)
elif NFRP_RA == 1 and TRA == 0 and individual_tones == 0:
    print('NFRP-RA enabled')
    print('Number of NDP Sets=', N)
        
print('<<--------------------------->>\n')

for num_UE in num_UE_g:
    if sample > 0:
        cobject = main(sample)
        
        print('# Number of UE:', num_UE)
        print('P_s:',cobject.P_s,'Da:',cobject.Da,'\n')
        
        print('Mi:',cobject.iteMi,'\n')
        print('MiSn:',cobject.iteMiSn,'\n')

        print('NiS:',cobject.iteNiS,'\n')
        print('NiI:',cobject.iteNiI,'\n')
    
print('end sim')

Number of samples= 100000
Number of RUs= 18
K= 1
Total transmissions= 5 

Features:
NFRP-RA enabled
Number of NDP Sets= 72
<<--------------------------->>

# Number of UE: 10
P_s: 1.0 Da: 1.337833779 

Mi: [[0, 0, 0, 0, 0], [10.0, 0.0, 0.0, 0.0, 0.0], [0.0, 1.18331, 0.0, 0.0, 0.0], [0.0, 0.0, 0.03014, 0.0, 0.0], [0.0, 0.0, 0.0, 0.00058, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0

In [17]:
sample = 0
CONFIG = 'UORA' #fixed-RA
ARRIVAL = 'one-shot'
R = 18 # number of RA-RU
K = 1
retrans = 4
num_UE_g = [10,20,30,40,50,60,70,80,90,100]
UORA_BO =  [7,15,31,31,31] #OCWmin~max [7,15,31,31,31]

# NFRP Features
NFRP_RA = 1
TRA = 0
individual_tones = 1

N = 4
NFRP_RA_BO = [7,15,31,31,31]

print('Number of samples=',sample)
print('Number of RUs=',R)
print('K=',K)
print('Total transmissions=', retrans+1,'\n')

print('Features:')
if NFRP_RA == 1 and TRA == 1 and individual_tones == 1:
    print('NFPR-enhanced UORA enabled')
    print('Number of NDP Sets=', N*2)
elif NFRP_RA == 1 and TRA == 1 and individual_tones == 0:
    print('NFPR-enhanced UORA enabled')
    print('Number of NDP Sets=', N)
elif NFRP_RA == 1 and TRA == 0 and individual_tones == 1:
    print('NFRP-RA enabled')
    print('Number of NDP Sets=', N*2)
elif NFRP_RA == 1 and TRA == 0 and individual_tones == 0:
    print('NFRP-RA enabled')
    print('Number of NDP Sets=', N)
        
print('<<--------------------------->>\n')

for num_UE in num_UE_g:
    if sample > 0:
        cobject = main(sample)
        
        print('# Number of UE:', num_UE)
        print('P_s:',cobject.P_s,'Da:',cobject.Da,'\n')
        
        print('Mi:',cobject.iteMi,'\n')
        print('MiSn:',cobject.iteMiSn,'\n')

        print('NiS:',cobject.iteNiS,'\n')
        print('NiI:',cobject.iteNiI,'\n')
    
print('end sim')

Number of samples= 100000
Number of RUs= 18
K= 1
Total transmissions= 5 

Features:
NFRP-RA enabled
Number of NDP Sets= 8
<<--------------------------->>

# Number of UE: 10
P_s: 0.999857 Da: 3.1806874303025325 

Mi: [[0, 0, 0, 0, 0], [10.0, 0.0, 0.0, 0.0, 0.0], [0.0, 3.92576, 0.0, 0.0, 0.0], [0.0, 3.06328, 0.40375, 0.0, 0.0], [0.0, 0.0, 0.63534, 0.03736, 0.0], [0.0, 0.0, 0.60343, 0.0495, 0.00121], [0.0, 0.0, 0.56314, 0.062, 0.00279], [0.0, 0.0, 0.21975, 0.07028, 0.0043], [0.0, 0.0, 0.0, 0.04021, 0.00522], [0.0, 0.0, 0.0, 0.02618, 0.00451], [0.0, 0.0, 0.0, 0.01269, 0.00343], [0.0, 0.0, 0.0, 0.00243, 0.00158], [0.0, 0.0, 0.0, 0.0, 0.00059], [0.0, 0.0, 0.0, 0.0, 0.00039], [0.0, 0.0, 0.0, 0.0, 0.00012], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0,

In [18]:
sample = 0
CONFIG = 'UORA' #fixed-RA
ARRIVAL = 'one-shot'
R = 18 # number of RA-RU
K = 1
retrans = 4
num_UE_g = [10,20,30,40,50,60,70,80,90,100]
UORA_BO =  [7,15,31,31,31] #OCWmin~max [7,15,31,31,31]

# NFRP Features
NFRP_RA = 1
TRA = 0
individual_tones = 1

N = 9
NFRP_RA_BO = [7,15,31,31,31]

print('Number of samples=',sample)
print('Number of RUs=',R)
print('K=',K)
print('Total transmissions=', retrans+1,'\n')

print('Features:')
if NFRP_RA == 1 and TRA == 1 and individual_tones == 1:
    print('NFPR-enhanced UORA enabled')
    print('Number of NDP Sets=', N*2)
elif NFRP_RA == 1 and TRA == 1 and individual_tones == 0:
    print('NFPR-enhanced UORA enabled')
    print('Number of NDP Sets=', N)
elif NFRP_RA == 1 and TRA == 0 and individual_tones == 1:
    print('NFRP-RA enabled')
    print('Number of NDP Sets=', N*2)
elif NFRP_RA == 1 and TRA == 0 and individual_tones == 0:
    print('NFRP-RA enabled')
    print('Number of NDP Sets=', N)
        
print('<<--------------------------->>\n')

for num_UE in num_UE_g:
    if sample > 0:
        cobject = main(sample)
        
        print('# Number of UE:', num_UE)
        print('P_s:',cobject.P_s,'Da:',cobject.Da,'\n')
        
        print('Mi:',cobject.iteMi,'\n')
        print('MiSn:',cobject.iteMiSn,'\n')

        print('NiS:',cobject.iteNiS,'\n')
        print('NiI:',cobject.iteNiI,'\n')
    
print('end sim')

Number of samples= 100000
Number of RUs= 18
K= 1
Total transmissions= 5 

Features:
NFRP-RA enabled
Number of NDP Sets= 18
<<--------------------------->>

# Number of UE: 10
P_s: 0.999999 Da: 1.8117417177417177 

Mi: [[0, 0, 0, 0, 0], [10.0, 0.0, 0.0, 0.0, 0.0], [0.0, 4.02951, 0.0, 0.0, 0.0], [0.0, 0.0, 0.46553, 0.0, 0.0], [0.0, 0.0, 0.31935, 0.01501, 0.0], [0.0, 0.0, 0.0, 0.01663, 0.00046], [0.0, 0.0, 0.0, 0.00475, 0.00067], [0.0, 0.0, 0.0, 0.0, 0.00027], [0.0, 0.0, 0.0, 0.0, 4e-05], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.